# Comparing Machine Learning Libraries

For this overview example, we will create a classification model using:

1. scikit-learn
2. Keras
3. PyTorch

## scikit-learn style

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split

In [2]:
%%time
cancer = load_breast_cancer()
X_scaled = StandardScaler().fit_transform(cancer.data)
print(X_scaled.shape)

(569, 30)
CPU times: user 15.7 ms, sys: 3.2 ms, total: 18.9 ms
Wall time: 33.6 ms


In [3]:
%%time
poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(X_scaled)
print(X_poly.shape)

(569, 496)
CPU times: user 14.7 ms, sys: 3.64 ms, total: 18.3 ms
Wall time: 43.1 ms


In [4]:
%%time
rfc = RandomForestClassifier(max_depth=7, n_estimators=10, random_state=1)
rfecv = RFECV(estimator=rfc, cv=5, n_jobs=-1)
X_poly_top = rfecv.fit_transform(X_poly, cancer.target)
print(X_poly_top.shape)

(569, 278)
CPU times: user 11.1 s, sys: 138 ms, total: 11.2 s
Wall time: 1min 7s


In [5]:
%%time
X_train, X_test, y_train, y_test = train_test_split(
    X_poly_top, cancer.target, random_state=42)

rfc = RandomForestClassifier(max_depth=7, n_estimators=10, random_state=1)
print(rfc.fit(X_train, y_train).score(X_test, y_test))

0.937062937063
CPU times: user 63.2 ms, sys: 5.19 ms, total: 68.4 ms
Wall time: 71.3 ms


## Keras style

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=42)

Using TensorFlow backend.


In [43]:
batch_size = 32
in_dim = cancer.data.shape[1]
hidden1 = X_poly_top.shape[1]
hidden2 = 20
out_dim = 1
batches_in_data = X_train.shape[0]/batch_size
epochs = int(5000/batches_in_data)
learning_rate = 1e-4


model_k = keras.models.Sequential([
    keras.layers.Dense(hidden1, activation='relu', input_shape=(in_dim,)),
    keras.layers.Dense(hidden2),
    keras.layers.LeakyReLU(),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(out_dim, activation='sigmoid')
])

In [44]:
model_k.summary()        

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 278)               8618      
_________________________________________________________________
dense_29 (Dense)             (None, 20)                5580      
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 20)                0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 21        
Total params: 14,219
Trainable params: 14,219
Non-trainable params: 0
_________________________________________________________________


In [45]:
model_k.compile(loss='mean_squared_error',
                optimizer=keras.optimizers.RMSprop(lr=learning_rate),
                metrics=['accuracy'])

history = model_k.fit(X_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=True,
                      validation_data=(X_test, y_test))
score = model_k.evaluate(X_test, y_test, verbose=True)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 426 samples, validate on 143 samples
Epoch 1/375
426/426 [==============================] - 0s - loss: 0.6313 - acc: 0.3685 - val_loss: 0.6224 - val_acc: 0.3776
Epoch 2/375
426/426 [==============================] - 0s - loss: 0.6246 - acc: 0.3756 - val_loss: 0.6224 - val_acc: 0.3776
Epoch 3/375
426/426 [==============================] - 0s - loss: 0.6240 - acc: 0.3756 - val_loss: 0.6224 - val_acc: 0.3776
Epoch 4/375
426/426 [==============================] - 0s - loss: 0.6103 - acc: 0.3826 - val_loss: 0.6223 - val_acc: 0.3776
Epoch 5/375
426/426 [==============================] - 0s - loss: 0.5357 - acc: 0.4601 - val_loss: 0.6224 - val_acc: 0.3776
Epoch 6/375
426/426 [==============================] - 0s - loss: 0.5165 - acc: 0.4718 - val_loss: 0.4019 - val_acc: 0.5315
Epoch 7/375
426/426 [==============================] - 0s - loss: 0.4576 - acc: 0.5282 - val_loss: 0.0850 - val_acc: 0.9021
Epoch 8/375
426/426 [==============================] - 0s - loss: 0.3832 - acc: 0.6080

426/426 [==============================] - 0s - loss: 0.0899 - acc: 0.9014 - val_loss: 0.0590 - val_acc: 0.9371
Epoch 132/375
426/426 [==============================] - 0s - loss: 0.0931 - acc: 0.9038 - val_loss: 0.0592 - val_acc: 0.9371
Epoch 133/375
426/426 [==============================] - 0s - loss: 0.0895 - acc: 0.9038 - val_loss: 0.0579 - val_acc: 0.9231
Epoch 134/375
426/426 [==============================] - 0s - loss: 0.0988 - acc: 0.8897 - val_loss: 0.0581 - val_acc: 0.9371
Epoch 135/375
426/426 [==============================] - 0s - loss: 0.0862 - acc: 0.9061 - val_loss: 0.0589 - val_acc: 0.9371
Epoch 136/375
426/426 [==============================] - 0s - loss: 0.0930 - acc: 0.9038 - val_loss: 0.0603 - val_acc: 0.9371
Epoch 137/375
426/426 [==============================] - 0s - loss: 0.0865 - acc: 0.9085 - val_loss: 0.0613 - val_acc: 0.9301
Epoch 138/375
426/426 [==============================] - 0s - loss: 0.0919 - acc: 0.8991 - val_loss: 0.0593 - val_acc: 0.9301
Epoch 

426/426 [==============================] - 0s - loss: 0.0726 - acc: 0.9155 - val_loss: 0.0415 - val_acc: 0.9510
Epoch 261/375
426/426 [==============================] - 0s - loss: 0.0741 - acc: 0.9131 - val_loss: 0.0543 - val_acc: 0.9301
Epoch 262/375
426/426 [==============================] - 0s - loss: 0.0737 - acc: 0.9225 - val_loss: 0.0434 - val_acc: 0.9441
Epoch 263/375
426/426 [==============================] - 0s - loss: 0.0804 - acc: 0.9061 - val_loss: 0.0424 - val_acc: 0.9510
Epoch 264/375
426/426 [==============================] - 0s - loss: 0.0772 - acc: 0.9108 - val_loss: 0.0601 - val_acc: 0.9231
Epoch 265/375
426/426 [==============================] - 0s - loss: 0.0779 - acc: 0.9202 - val_loss: 0.0503 - val_acc: 0.9441
Epoch 266/375
426/426 [==============================] - 0s - loss: 0.0820 - acc: 0.9038 - val_loss: 0.0400 - val_acc: 0.9510
Epoch 267/375
426/426 [==============================] - 0s - loss: 0.0723 - acc: 0.9202 - val_loss: 0.0408 - val_acc: 0.9441
Epoch 

## PyTorch style

In [10]:
import numpy as np
import torch
from torch.autograd import Variable

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=42)
cancer.data.shape

(569, 30)

In [11]:
# Create a sequential NN
batch_size = 32
in_dim = cancer.data.shape[1]
hidden1 = X_poly_top.shape[1]
hidden2 = 20
out_dim = 1
learning_rate = 1e-4

model_t = torch.nn.Sequential(
    torch.nn.Linear(in_dim, hidden1),
    torch.nn.ReLU(),
    # Add a hidden layer that loosely represents the poly features
    torch.nn.Linear(hidden1, hidden2),
    torch.nn.LeakyReLU(),  ## Use LeakyReLU might limit dead neurons
    # A Dropout layer sometimes reduces co-adaptation of neurons
    torch.nn.Dropout(p=0.25),
    # Add a second hidden layer for further abstraction
    torch.nn.Linear(hidden2, out_dim),  
    # Add an output layer
    torch.nn.Sigmoid()
)

loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.RMSprop(model_t.parameters(), lr=learning_rate)

In [12]:
from torchsummary import summary
summary(model_t, input_size=(1,in_dim))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 278]           8,618
              ReLU-2               [-1, 1, 278]               0
            Linear-3                [-1, 1, 20]           5,580
         LeakyReLU-4                [-1, 1, 20]               0
           Dropout-5                [-1, 1, 20]               0
            Linear-6                 [-1, 1, 1]              21
           Sigmoid-7                 [-1, 1, 1]               0
Total params: 14,219
Trainable params: 14,219
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.05
Estimated Total Size (MB): 0.06
----------------------------------------------------------------


In [33]:
model_k.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 278)               8618      
_________________________________________________________________
dense_14 (Dense)             (None, 20)                5580      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 20)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 21        
Total params: 14,219
Trainable params: 14,219
Non-trainable params: 0
_________________________________________________________________


In [13]:
## Now run model
X = torch.from_numpy(X_train).float()
y = torch.from_numpy(y_train[:, np.newaxis]).float()
X_test_T = torch.from_numpy(X_test).float()
y_test_T = torch.from_numpy(y_test[:, np.newaxis]).float()

show_every = 75
for t in range(5000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model_t(X)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if not t % show_every:
        y_test_pred = model_t(Variable(X_test_T))
        prediction = [int(x > 0.5) for x in y_test_pred.data.numpy()]
        test_accuracy = (prediction == y_test).sum() / len(y_test)
        train_pred = [int(x > 0.5) for x in y_pred.data.numpy()]
        train_accuracy = (train_pred == y_train).sum() / len(y_train)
        print("Batch: %04d | Training Loss: %6.2f | Train accuracy: %.4f | Test accuracy: %.4f" % (
                      t, loss.item(), train_accuracy, test_accuracy))

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

Batch: 0000 | Training Loss: 246.87 | Train accuracy: 0.3967 | Test accuracy: 0.4266
Batch: 0075 | Training Loss: 148.80 | Train accuracy: 0.6479 | Test accuracy: 0.6364
Batch: 0150 | Training Loss: 150.97 | Train accuracy: 0.6432 | Test accuracy: 0.6434
Batch: 0225 | Training Loss: 149.40 | Train accuracy: 0.6502 | Test accuracy: 0.6503
Batch: 0300 | Training Loss: 143.48 | Train accuracy: 0.6620 | Test accuracy: 0.6224
Batch: 0375 | Training Loss:  33.58 | Train accuracy: 0.9108 | Test accuracy: 0.9580
Batch: 0450 | Training Loss:  28.26 | Train accuracy: 0.9202 | Test accuracy: 0.9510
Batch: 0525 | Training Loss:  27.62 | Train accuracy: 0.9202 | Test accuracy: 0.9441
Batch: 0600 | Training Loss:  29.57 | Train accuracy: 0.9108 | Test accuracy: 0.9510
Batch: 0675 | Training Loss:  25.97 | Train accuracy: 0.9296 | Test accuracy: 0.9371
Batch: 0750 | Training Loss:  24.36 | Train accuracy: 0.9272 | Test accuracy: 0.9510
Batch: 0825 | Training Loss:  24.89 | Train accuracy: 0.9202 | Te